In [ ]:
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode

!python -m pip install nemo_toolkit[all]

exit()

In [ ]:
# Папка, где будет размещаться датасет Golos
data_dir = '/content/farfield'

In [ ]:
import glob
import os
import subprocess
import tarfile
import wget

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr

[NeMo W 2021-11-28 16:51:00 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join("/content/QuartzNet15x5_golos.nemo"))

[NeMo W 2021-11-28 16:59:40 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-11-28 16:59:40 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2021-11-28 16:59:40 features:265] PADDING: 16
[NeMo I 2021-11-28 16:59:40 features:282] STFT using torch
[NeMo I 2021-11-28 16:59:52 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/QuartzNet15x5_golos.nemo.


Следующая ниже часть выполнялась локально, поэтому пути и проч. не соответствуют тем, что в colab.

Подсчитаем количество файлов, которые предстоит затранскрибировать.

In [ ]:
from os import listdir
from os.path import isfile, join
print(data_dir)
onlyfiles = [data_dir + '/' + f for f in listdir(data_dir) if isfile(join(data_dir, f))]
print(len(onlyfiles))
#farfield
#124003

Приступаем к транскрибированию с шагом 500. Если попадается битый файл, это не так критично. После каждых 500 транскрипций результат сохраняется в *transcription_summary.txt*.

In [ ]:
for x in range(0, len(onlyfiles))[::500]:
  if x + 500 < len(onlyfiles) - 1:
    y = x + 500
  else:
    y = -1
  try:
    transcriptions = asr_model.transcribe(paths2audio_files=onlyfiles[x:y])
    with open(f'transcription_summary.txt', 'a+', encoding='UTF-8') as ff:
      for fname, transcription in zip(onlyfiles[x:y], transcriptions):
        file1 = fname.split('/')[-1]
        ff.write(f'{file1},{transcription}\n')
  except Exception as e:
    print(e, f'{x}:{y}', 'mist_ake')
    continue

Теперь нужно посчитать расстояние Левенштейна, обновить пути к аудио, а также добавить текущие транскрипции, которые делает наша модель.

In [ ]:
import Levenshtein

manifest = 'manifest.jsonl'
audio = '/content'

with open('all_trans.txt', 'r') as f:
    trans = f.readlines()

all_trans = {}
for t in trans:
    t = t.split(',')
    all_trans.update({t[0].replace('.opus', ''): t[1]})

with open(manifest, 'r') as fin:
    with open('new_manifest.txt', 'w') as fout:
        for line in fin:
            metadata = json.loads(line)
            metadata["audio_filepath"]=os.path.join(audio, metadata["audio_filepath"])
            trans_current=all_trans.get(metadata["id"])
            if not trans_current:
                continue
            metadata["trans_current"]=trans_current
            metadata["lev"]=Levenshtein.distance(metadata["text"], trans_current)
            if metadata["lev"] in [0, 1, 2] or metadata["lev"] > 40:
                continue
            json.dump(metadata, fout)
            fout.write('\n')

Как видно, в процессе также отсеиваем идентичные и почти идентичные транскрипции, а также выбросы со слишком большим расстоянием. Теперь, проделав аналогичную операцию с манифестом для тестирования -- только там меняем исключительно пути -- можно проконтролировать данные с помощью *data_analyzer.py*.

In [ ]:
test_man = 'test_farfield.jsonl'
audio = '/content/test_farfield'
with open(test_man, 'r') as fin:
    with open('new_test_manifest.txt', 'w') as fout:
        for line in fin:
            metadata = json.loads(line)
            metadata["audio_filepath"]=os.path.join(audio, metadata["audio_filepath"].replace('files/', ''))
            json.dump(metadata, fout)
            fout.write('\n')

Результаты промежуточных операций: [new_manifest.txt](https://drive.google.com/file/d/1a6W3ysrW49eiBkBCoyi-RPvaHNTLHq8f/view?usp=sharing), [transcription_summary.txt](https://drive.google.com/file/d/1PyVgKKlHd8ZDOU5OWrlmfuUh070yoj4h/view?usp=sharing).

Переходим к обучению модели в colab.

In [ ]:
test_manifest = '/content/new_test_manifest.txt'
train_manifest = '/content/new_manifest.txt'

In [ ]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=10, precision=16)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
params_model = asr_model._cfg
params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size']=16
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16

In [ ]:
params_model

{'sample_rate': 16000, 'repeat': 5, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'optim': {'name': 'novograd', 'lr': 0.0001, 'betas': [0.9, 0.98], 'weight_decay': 0.001, 'sched': {'name': 'CosineAnnealing', 'warmup_steps': 500, 'warmup_ratio': None, 'min_lr': 0.0, 'last_epoch': -1}}, 'train_ds': {'manifest_filepath': '/content/new_manifest.txt', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 16, 'trim_silence': False, 'max_duration': 20.0, 'min_duration': 0.1, 'num_workers': 20, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'tarred_shard_strategy': 'scatter', 'parser': 'base'}, 'validation_ds': {'manifest_filepath': '/content/new_test_m

In [ ]:
params_model['optim']['lr'] = 0.0005
print(params_model['optim']['lr'])

0.0005


In [ ]:
from omegaconf import DictConfig
first_asr_model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params_model), trainer=trainer)

[NeMo I 2021-11-28 18:01:34 collections:173] Dataset loaded with 1239 files totalling 1.29 hours
[NeMo I 2021-11-28 18:01:34 collections:174] 2 files were filtered totalling 0.01 hours


[NeMo W 2021-11-28 18:01:34 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-28 18:01:34 collections:173] Dataset loaded with 546 files totalling 0.40 hours
[NeMo I 2021-11-28 18:01:34 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-28 18:01:34 collections:173] Dataset loaded with 546 files totalling 0.40 hours
[NeMo I 2021-11-28 18:01:34 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-28 18:01:34 features:265] PADDING: 16
[NeMo I 2021-11-28 18:01:34 features:282] STFT using torch


In [ ]:
trainer.fit(first_asr_model)

In [ ]:
print(params_model['optim'])

{'name': 'novograd', 'lr': 0.0005, 'betas': [0.9, 0.98], 'weight_decay': 0.001, 'sched': {'name': 'CosineAnnealing', 'warmup_steps': 500, 'warmup_ratio': None, 'min_lr': 0.0, 'last_epoch': -1}}


Попробуем улучшить положение дел. Текущий WER -- 0.6862305494011274. Повысим LR в два раза для fine-tuning'а.

In [ ]:
import copy
new_opt = copy.deepcopy(params_model['optim'])
new_opt['lr'] = 0.001
first_asr_model.setup_optimization(optim_config=DictConfig(new_opt))

[NeMo W 2021-11-28 19:44:44 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7f84ef711ad0>. We recommend to use 'ddp' instead.


[NeMo I 2021-11-28 19:44:44 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )
[NeMo I 2021-11-28 19:44:44 lr_scheduler:752] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f84ef61e2d0>" 
    will be used during training (effective maximum steps = 780) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 780
    )


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.9, 0.98]
     eps: 1e-08
     grad_averaging: False
     initial_lr: 0.001
     lr: 1.996007984031936e-06
     weight_decay: 0.001
 ),
 {'frequency': 1,
  'interval': 'step',
  'monitor': 'loss',
  'reduce_on_plateau': False,
  'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7f84ef61e2d0>})

In [ ]:
first_asr_model.setup_training_data(train_data_config=params_model['train_ds'])
first_asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

trainer = pl.Trainer(gpus=1, max_epochs=10, precision=16)
trainer.fit(first_asr_model)

[NeMo I 2021-11-28 19:48:13 collections:173] Dataset loaded with 1239 files totalling 1.29 hours
[NeMo I 2021-11-28 19:48:13 collections:174] 2 files were filtered totalling 0.01 hours


[NeMo W 2021-11-28 19:48:13 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-28 19:48:13 collections:173] Dataset loaded with 546 files totalling 0.40 hours
[NeMo I 2021-11-28 19:48:13 collections:174] 0 files were filtered totalling 0.00 hours


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-28 19:48:13 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7f84ef711ad0>. We recommend to use 'ddp' instead.


[NeMo I 2021-11-28 19:48:13 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )
[NeMo I 2021-11-28 19:48:13 lr_scheduler:752] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f850bcc54d0>" 
    will be used during training (effective maximum steps = 780) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 780
    )


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir lightning_logs/
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [3]:
first_asr_model.cuda()
wer_nums = []
wer_denoms = []

for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions
        
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-28 21:41:16 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1

WER = 0.5765447494011353


WER понизился, эффект от повышения LR есть. Я не уверен, что это хороший показатель всё равно, потому что не смог найти ориентиров для нормального WER, хотя пытался. Более того, наше задание немного "игрушечно", поэтому, наверно, эта цифра подойдет. Важно то, что по сравнению с первой попыткой WER уменьшился.

**WER = 0.5765447494011353**